In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ffipy import FFIEC_Client
from io import StringIO

In [27]:
failed_banks_df = pd.read_csv('data/failed_banks_dataset.csv')
failed_banks_df['Date'] = pd.to_datetime(failed_banks_df['Closing Date'])
failed_banks_df['Year_failed'] = failed_banks_df['Date'].dt.year
failed_banks_df['Quarter_failed'] = failed_banks_df['Date'].dt.quarter
failed_banks_2010Q4 = failed_banks_df.loc[(failed_banks_df.Year_failed == 2010) & (failed_banks_df.Quarter_failed == 4)]
failed_banks_2010Q4.head()
failed_banks_2010Q4['CERT'].count()

30

In [28]:
# Prepare dictionary of selected financials that are needed for CAMELS
selected_financials = {
                       'RIAD3210' : 'Total equity capital', # Alt can be RCONG105
                       'RCON2170' : 'Total assets', # Ok
                       'RCON3360' : 'Total loans', #ok
                       'RCON3465' : '1-4 family residential loans', 
                       'RCON3466' : 'Other real estate loans',
                       'RCON3387' : 'Commercial and industrial loans',
                       'RCONB561' : 'Credit cards',
                       'RCON3123' : 'Allowance for loan losses', # OK
                       'RIAD4093' : 'Total noninterest expense', #OK
                       'RIAD4300' : 'Net Income before', # # new for healthy
                       'RCON2215' : 'Total transaction deposits', # ok
                       'RCON2385' : 'Total nontransaction deposits', # ok
                       'RCON1773' : 'Available-for-sale Fair Value',
                       'RIAD4150' : 'Number of full-time employees'# new for healthy
                      } 

In [29]:
selected_financials

{'RIAD3210': 'Total equity capital',
 'RCON2170': 'Total assets',
 'RCON3360': 'Total loans',
 'RCON3465': '1-4 family residential loans',
 'RCON3466': 'Other real estate loans',
 'RCON3387': 'Commercial and industrial loans',
 'RCONB561': 'Credit cards',
 'RCON3123': 'Allowance for loan losses',
 'RIAD4093': 'Total noninterest expense',
 'RIAD4300': 'Net Income before',
 'RCON2215': 'Total transaction deposits',
 'RCON2385': 'Total nontransaction deposits',
 'RCON1773': 'Available-for-sale Fair Value',
 'RIAD4150': 'Number of full-time employees'}

In [30]:
len(selected_financials.keys())
columns = list(selected_financials.keys())
len(columns)

14

In [47]:
banks_to_pull = list(failed_banks_2010Q4['CERT'])


In [32]:
client = FFIEC_Client()

In [48]:
# Pull 30 reports, drop all values but 13 selected metrics -- returns only 27
report_data_dict = {}
for i in range(len(banks_to_pull)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'FDICCertNumber'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = banks_to_pull[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling CERT# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        
        key = banks_to_pull[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        
        
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
        #values = list(report_df[report_df.ValueID.isin(columns)].Value)
        #fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

counter at  0  pulling CERT#  23306
counter at  1  pulling CERT#  58052
counter at  2  pulling CERT#  35065
counter at  3  pulling CERT#  58495
counter at  4  pulling CERT#  34578
counter at  5  pulling CERT#  19040
counter at  6  pulling CERT#  35561
counter at  7  pulling CERT#  34673
counter at  8  pulling CERT#  5287
counter at  9  pulling CERT#  35078
counter at  10  pulling CERT#  20340
counter at  11  pulling CERT#  35463
counter at  12  pulling CERT#  14580
counter at  13  pulling CERT#  57831
counter at  14  pulling CERT#  57885
counter at  15  pulling CERT#  34411
counter at  16  pulling CERT#  58087
counter at  17  pulling CERT#  31263
counter at  18  pulling CERT#  32582
counter at  19  pulling CERT#  22173
counter at  20  pulling CERT#  16089
counter at  21  pulling CERT#  2119
counter at  22  pulling CERT#  33904
counter at  23  pulling CERT#  32251
counter at  24  pulling CERT#  27573
counter at  25  pulling CERT#  34016
counter at  26  pulling CERT#  58205
counter at  2

In [54]:
failed_after_2010Q3 = pd.DataFrame.from_dict(report_data_dict, orient='index')
failed_after_2010Q3.columns = selected_financials.values()

In [55]:
#failed_after_2010Q3.to_csv('data/failed_after_2010Q3')


In [103]:
failed_banks_2010Q3 = failed_banks_df.loc[(failed_banks_df.Year_failed == 2010) & (failed_banks_df.Quarter_failed == 3)]
banks_CERT_2010Q3_list = list(failed_banks_2010Q3.CERT)
banks_to_pull= list(banks_CERT_2010Q3_list)

In [104]:
# Pull 41 reports, drop all values but 13 selected metrics 
report_data_dict = {}
for i in range(len(banks_to_pull)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '6/30/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'FDICCertNumber'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = banks_to_pull[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling CERT# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        
        key = banks_to_pull[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        
        
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
        #values = list(report_df[report_df.ValueID.isin(columns)].Value)
        #fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

counter at  0  pulling CERT#  35053
counter at  1  pulling CERT#  58308
counter at  2  pulling CERT#  28612
counter at  3  pulling CERT#  27808
counter at  4  pulling CERT#  182
counter at  5  pulling CERT#  57448
counter at  6  pulling CERT#  58197
counter at  7  pulling CERT#  57107
counter at  8  pulling CERT#  35061
counter at  9  pulling CERT#  27259
counter at  10  pulling CERT#  32165
counter at  11  pulling CERT#  33219
counter at  12  pulling CERT#  27090
counter at  13  pulling CERT#  15640
counter at  14  pulling CERT#  31623
counter at  15  pulling CERT#  27344
counter at  16  pulling CERT#  25266
counter at  17  pulling CERT#  17599
counter at  18  pulling CERT#  34231
counter at  19  pulling CERT#  31964
counter at  20  pulling CERT#  22643
counter at  21  pulling CERT#  9619
counter at  22  pulling CERT#  57669
counter at  23  pulling CERT#  57658
counter at  24  pulling CERT#  23181
counter at  25  pulling CERT#  35434
counter at  26  pulling CERT#  34486
counter at  27

In [105]:
failed_after_2010Q2 = pd.DataFrame.from_dict(report_data_dict, orient='index')
failed_after_2010Q2.columns = selected_financials.values()

In [106]:
#failed_after_2010Q2.to_csv('data/failed_after_2010Q2')

In [107]:
failed_banks_2010Q2 = failed_banks_df.loc[(failed_banks_df.Year_failed == 2010) & (failed_banks_df.Quarter_failed == 2)]
banks_CERT_2010Q2_list = list(failed_banks_2010Q2.CERT)
banks_to_pull = banks_CERT_2010Q2_list
banks_to_pull[44]

34242

In [108]:
# Pull 45 reports, drop all values but 13 selected metrics 
report_data_dict = {}
for i in range(len(banks_to_pull)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '3/31/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'FDICCertNumber'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = banks_to_pull[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling CERT# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        
        key = banks_to_pull[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        
        
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
        #values = list(report_df[report_df.ValueID.isin(columns)].Value)
        #fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

counter at  0  pulling CERT#  35279
counter at  1  pulling CERT#  34152
counter at  2  pulling CERT#  26563
counter at  3  pulling CERT#  57110
counter at  4  pulling CERT#  32955
counter at  5  pulling CERT#  29341
counter at  6  pulling CERT#  31813
counter at  7  pulling CERT#  15814
counter at  8  pulling CERT#  34785
counter at  9  pulling CERT#  57315
counter at  10  pulling CERT#  57814
counter at  11  pulling CERT#  35106
counter at  12  pulling CERT#  57360
counter at  13  pulling CERT#  57735
counter at  14  pulling CERT#  18117
counter at  15  pulling CERT#  34255
counter at  16  pulling CERT#  35586
counter at  17  pulling CERT#  35114
counter at  18  pulling CERT#  35517
counter at  19  pulling CERT#  57697
counter at  20  pulling CERT#  16476
counter at  21  pulling CERT#  14246
counter at  22  pulling CERT#  22710
counter at  23  pulling CERT#  17792
counter at  24  pulling CERT#  58362
counter at  25  pulling CERT#  30005
counter at  26  pulling CERT#  31027
counter at 

In [109]:
failed_after_2010Q1 = pd.DataFrame.from_dict(report_data_dict, orient='index')
failed_after_2010Q1.columns = selected_financials.values()

In [110]:
failed_after_2010Q1.tail()

,Total equity capital,Total assets,Total loans,1-4 family residential loans,Other real estate loans,Commercial and industrial loans,Credit cards,Allowance for loan losses,Total noninterest expense,Net Income before,Total transaction deposits,Total nontransaction deposits,Available-for-sale Fair Value,Number of full-time employees
57724,-317,104034,65031,7104,52235,5304,0,3813,936,-684,6153,93917,4024,20
57735,-446,61215,58810,15417,27911,10969,0,5182,684,-1205,4515,53773,1000,12
57814,-2177,240513,210642,11468,134021,24768,0,9253,2629,-9449,5650,218374,24770,23
58362,3469,195510,139262,31823,100400,5887,0,8025,1611,-765,13303,149513,11670,29
58429,-10986,441694,396899,46353,292970,51370,0,34827,3224,-2953,6038,442879,2367,23


In [111]:
#failed_after_2010Q1.to_csv('data/failed_after_2010Q1')

In [112]:
failed_banks_2010Q1 = failed_banks_df.loc[(failed_banks_df.Year_failed == 2010) & (failed_banks_df.Quarter_failed == 1)]
banks_CERT_2010Q1_list = list(failed_banks_2010Q1.CERT)
banks_to_pull = banks_CERT_2010Q1_list
banks_to_pull[40]

22977

In [113]:
# Pull 41 reports, drop all values but 13 selected metrics < -- returns 37
report_data_dict = {}
for i in range(len(banks_to_pull)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '12/31/2009'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'FDICCertNumber'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = banks_to_pull[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling CERT# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        
        key = banks_to_pull[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        
        
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
        #values = list(report_df[report_df.ValueID.isin(columns)].Value)
        #fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

counter at  0  pulling CERT#  57060
counter at  1  pulling CERT#  34678
counter at  2  pulling CERT#  34684
counter at  3  pulling CERT#  57399
counter at  4  pulling CERT#  8221
counter at  5  pulling CERT#  24957
counter at  6  pulling CERT#  10054
counter at  7  pulling CERT#  33989
counter at  8  pulling CERT#  33535
counter at  9  pulling CERT#  58104
counter at  10  pulling CERT#  18806
counter at  11  pulling CERT#  29561
counter at  12  pulling CERT#  58182
counter at  13  pulling CERT#  27096
counter at  14  pulling CERT#  58071
counter at  15  pulling CERT#  34430
counter at  16  pulling CERT#  34976
counter at  17  pulling CERT#  9268
counter at  18  pulling CERT#  27126
counter at  19  pulling CERT#  38129
counter at  20  pulling CERT#  58352
counter at  21  pulling CERT#  32423
counter at  22  pulling CERT#  29952
counter at  23  pulling CERT#  3287
counter at  24  pulling CERT#  57586
counter at  25  pulling CERT#  15448
counter at  26  pulling CERT#  16730
counter at  27

In [114]:
failed_after_2009Q4 = pd.DataFrame.from_dict(report_data_dict, orient='index')
failed_after_2009Q4.columns = selected_financials.values()

In [115]:
#failed_after_2009Q4.to_csv('data/failed_after_2009Q4')

In [ ]:
# TODO

In [199]:
# after 2009Q3

In [200]:
# after 2009Q2

In [201]:
# after 2009Q1

In [202]:
# after 2008Q4

In [203]:
# after 2008Q3

In [204]:
# after 2008Q2

In [ ]:
# after 2008Q1